In [1]:
import polars as pl
import torch

In [2]:
device = 'cuda'
torch.set_default_device(device)

In [3]:
def check_cuda_memory():
    if torch.cuda.is_available():
        # Get total memory in bytes
        total_memory = torch.cuda.get_device_properties(0).total_memory
        
        # Get allocated memory in bytes
        allocated_memory = torch.cuda.memory_allocated(0)
        
        # Get cached memory in bytes (reserved by PyTorch but not used)
        cached_memory = torch.cuda.memory_reserved(0)
        
        # Calculate free memory
        free_memory = total_memory - allocated_memory - cached_memory
        
        # Convert to more readable units (GB)
        total_gb = total_memory / 1e9
        allocated_gb = allocated_memory / 1e9
        cached_gb = cached_memory / 1e9
        free_gb = free_memory / 1e9
        
        print(f"CUDA Memory Status:")
        print(f"Total: {total_gb:.2f} GB")
        print(f"Allocated: {allocated_gb:.2f} GB ({allocated_gb/total_gb*100:.1f}%)")
        print(f"Cached: {cached_gb:.2f} GB ({cached_gb/total_gb*100:.1f}%)")
        print(f"Free: {free_gb:.2f} GB ({free_gb/total_gb*100:.1f}%)")
    else:
        print("CUDA is not available on this system")

# Call the function to check memory
check_cuda_memory()

CUDA Memory Status:
Total: 6.10 GB
Allocated: 0.00 GB (0.0%)
Cached: 0.00 GB (0.0%)
Free: 6.10 GB (100.0%)


In [4]:
torch.cuda.empty_cache()

In [5]:
data = pl.read_csv('data/merchant_locs_train.csv', separator=',', quote_char='"').sample(10000)

In [6]:
data['online_offline_flag'].value_counts()

online_offline_flag,count
str,u32
"""ONLINE""",3417
"""OFFLINE""",6583


In [7]:
data

source,merchant_name,merchant_location,online_offline_flag
str,str,str,str
"""CREDIT_CARD""","""TRAVELOKA-1223435704 JAKARTA I…",null,"""ONLINE"""
"""QRIS_WONDR""","""PECEL LELE AYAM KREMES MA""","""BOGOR""","""OFFLINE"""
"""QRIS_WONDR""","""MAKAM SYEKH AHMAD MUTAMAK""","""PATI""","""OFFLINE"""
"""QRIS_WONDR""","""KEDAI HENOT""","""KOTA PALEMBANG""","""OFFLINE"""
"""QRIS_WONDR""","""PKM DUKUH KUPANG""","""SURABAYA""","""OFFLINE"""
…,…,…,…
"""QRIS_WONDR""","""MERAH PUTIH LAUNDRY FA...""","""SURAKARTA""","""OFFLINE"""
"""QRIS_WONDR""","""BUBUR AYAM SANTANI""","""BOGOR""","""OFFLINE"""
"""QRIS_WONDR""","""CINTA DAMAI PUTRA BAHAGIA""","""MAJALENGKA (K""","""OFFLINE"""


In [7]:
mapping = {"ONLINE": 1, "OFFLINE": 0}

In [8]:
data = data.with_columns(
    pl.col("online_offline_flag").replace_strict(mapping).alias("label")
)

In [9]:
import re
def count_digit(x):
    return sum(c.isdigit() for c in x)

data = data.with_columns(
    pl.col("merchant_name").map_elements(lambda x: count_digit(x)/len(x), return_dtype = pl.Float32).alias("digit_ratio"),
    pl.col("merchant_name").map_elements(lambda x: len(re.findall(r'\d+',x)), return_dtype = pl.Int32).alias("num_blocks"),
    #pl.when(pl.col("merchant_location").is_null()).then(pl.lit(0)).otherwise(pl.lit(1)).alias("location_is_known"),
    pl.when(pl.col("merchant_name").str.contains(r"((\bAFM\b)|(ALFAMART))|((\bIDM\b|INDOMARET))|(FAMILY\s?MART)|(HHB\s[A-Z0-9]{3})")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("known_offline_merchant")
)

In [10]:
data

source,merchant_name,merchant_location,online_offline_flag,label,digit_ratio,num_blocks,known_offline_merchant
str,str,str,str,i64,f32,i32,i32
"""QRIS_WONDR""","""YUN CELL HO""","""DENPASAR""","""OFFLINE""",0,0.0,0,0
"""CREDIT_CARD""","""SHOP TKPD 6450626132 JAKARTA I…",null,"""ONLINE""",1,0.322581,1,0
"""QRIS_WONDR""","""HENDRA MART LOKTUAN 2""","""BONTANG KOT.""","""OFFLINE""",0,0.047619,1,0
"""QRIS_WONDR""","""MARTABAK BANGKA LUNGA TEK""","""JAKARTA""","""OFFLINE""",0,0.0,0,0
"""CREDIT_CARD""","""EDAS CONF* ICTED2024 H LEONIA …",null,"""ONLINE""",1,0.125,1,0
…,…,…,…,…,…,…,…
"""QRIS_WONDR""","""TOKO MUSLIM""","""TANJUNG PINANG""","""OFFLINE""",0,0.0,0,0
"""QRIS_WONDR""","""BARBER SHOPS APRII""","""TULANGBAWANG BA""","""OFFLINE""",0,0.0,0,0
"""QRIS_WONDR""","""ADVENTURE KOPI""","""BOGOR KAB.""","""OFFLINE""",0,0.0,0,0


In [11]:
from sklearn.model_selection import train_test_split
X = data[['merchant_name','digit_ratio', 'num_blocks', #'location_is_known', 
          'known_offline_merchant']]
y = data['label']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [13]:
X_test_raw

merchant_name,digit_ratio,num_blocks,known_offline_merchant
str,f32,i32,i32
"""LAB CITO BOGOR""",0.0,0,0
"""INDOMARET TU67 SALATIGA""",0.086957,1,1
"""TRAVELOKA3DS*1210524667 JAKART…",0.323529,2,0
"""-TKP292496391JAKARTA ID""",0.391304,1,0
"""TRAVELOKAC*1216941198 JAKARTA …",0.3125,1,0
…,…,…,…
"""EBAY O*08-12584-27572 408-3766…",0.611111,5,0
"""-TKP292218179JAKARTA ID""",0.391304,1,0
"""BATIK ATIK.""",0.0,0,0


In [14]:
# bert auto tokenizer
import transformers, tqdm

/home/master/work/nn_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
bert_model = transformers.AutoModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

In [16]:
bert_model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 312, padding_idx=0)
    (position_embeddings): Embedding(512, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-3): 4 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [17]:
bert_input = bert_tokenizer(list(X_train_raw['merchant_name']), return_tensors="pt", padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
# Get embeddings
with torch.no_grad():
    bert_output = bert_model(**bert_input)

In [19]:
bert_feature = bert_output.last_hidden_state[:, 0, :]

In [20]:
bert_feature.shape

torch.Size([7000, 312])

In [21]:
# preprocessing for tfidf: remove special characters, numbers, extra space, convert to lowercase
import re

def clean_merchant_name(x):
    x1 = re.sub(r'[^a-zA-Z\s]', '', x)
    x2 = re.sub(r'\s+', ' ', x1).lower()
    return x2

data = data.with_columns(
    pl.col("merchant_name").map_elements(lambda x: clean_merchant_name(x), return_dtype = pl.Utf8).alias("merchant_name_clean")
)

In [184]:
X = data['merchant_name_clean']
y = data['label']

In [189]:
from sklearn.model_selection import train_test_split

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [200]:
# sklearn TFIDF tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='char_wb',  # Character n-grams including word boundaries
    ngram_range=(2, 5),  # 2-4 character sequences
    min_df=4,
    max_df=0.95,
    sublinear_tf=True, 
    norm='l2',  
    use_idf=True, 
    smooth_idf=True, 
)
X_train = vectorizer.fit_transform(X_train_raw)

In [198]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 19484934 stored elements and shape (350000, 119182)>

In [201]:
X_test = vectorizer.transform(X_test_raw)

In [202]:
X_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8325890 stored elements and shape (150000, 119182)>

In [82]:
# modeling with torch

In [22]:
import torch

In [23]:
device = 'cuda'
torch.set_default_device(device)

In [67]:
X_train.indptr
X_train.indices

array([22129, 42212,  2633, ..., 45516, 28812, 38533],
      shape=(3260473,), dtype=int32)

In [72]:
X_train_csr_tensor = torch.sparse_csr_tensor(torch.from_numpy(X_train.indptr),
                                       torch.from_numpy(X_train.indices),
                                       torch.from_numpy(X_train.data),
                                        dtype = torch.float32)

In [77]:
X_test_csr_tensor = torch.sparse_csr_tensor(torch.from_numpy(X_test.indptr),
                                       torch.from_numpy(X_test.indices),
                                       torch.from_numpy(X_test.data),
                                        dtype = torch.float32)

In [81]:
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [125]:
# baseline model, xgboost

In [230]:
X_train_raw

merchant_name,digit_ratio,num_blocks,known_offline_merchant
str,f32,i32,i32
"""TOKO PALL""",0.0,0,0
"""TRAVELOKA*1221658246 JAKARTA I…",0.322581,1,0
"""SEKIAN KOPI PORIS, CIPOND""",0.0,0,0
"""RM PADANG BASALERO""",0.0,0,0
"""WARUNG IJO BPK. WAHYU""",0.0,0,0
…,…,…,…
"""SHOP TKPD 6092057227 JAKARTA I…",0.322581,1,0
"""CAHAYA SIDRAP BIRD SHOP""",0.0,0,0
"""FLIPSIDE TEXNO""",0.0,0,0


In [26]:
X_train = torch.cat([torch.tensor(X_train_raw[['digit_ratio', 'num_blocks', 
                                               #'location_is_known',
                                               'known_offline_merchant']].to_numpy()).to(device), 
                     #bert_feature
                    ], dim=1)
y_train = torch.tensor(y_train)

In [27]:
X_train.shape

torch.Size([7000, 3])

In [28]:
# TRAINING

In [29]:
test_bert_input = bert_tokenizer(list(X_test_raw['merchant_name']), return_tensors="pt", padding=True, truncation=True)
# Get embeddings
with torch.no_grad():
    test_bert_output = bert_model(**test_bert_input)

In [30]:
test_bert_feature = test_bert_output.last_hidden_state[:, 0, :]

In [31]:
X_test = torch.cat([torch.tensor(X_test_raw[['digit_ratio','num_blocks',
                                             #'location_is_known',
                                             'known_offline_merchant']].to_numpy()).to(device), 
                    #test_bert_feature
                   ], 
                   dim=1)
y_test = torch.tensor(y_test)

In [32]:
import xgboost as xgb
model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=500,
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',  # For faster training with sparse data
    device='cuda',
    random_state = 1,
    early_stopping_rounds = 50
)

In [33]:
# Train model
model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True,
)

[0]	validation_0-logloss:0.56847
[1]	validation_0-logloss:0.51730
[2]	validation_0-logloss:0.47502
[3]	validation_0-logloss:0.44053
[4]	validation_0-logloss:0.41153
[5]	validation_0-logloss:0.37399
[6]	validation_0-logloss:0.34207
[7]	validation_0-logloss:0.31467
[8]	validation_0-logloss:0.29421
[9]	validation_0-logloss:0.28035
[10]	validation_0-logloss:0.26835
[11]	validation_0-logloss:0.25358
[12]	validation_0-logloss:0.23754
[13]	validation_0-logloss:0.22956
[14]	validation_0-logloss:0.21908
[15]	validation_0-logloss:0.21300
[16]	validation_0-logloss:0.20194
[17]	validation_0-logloss:0.19213
[18]	validation_0-logloss:0.18339
[19]	validation_0-logloss:0.17719
[20]	validation_0-logloss:0.17165
[21]	validation_0-logloss:0.16542
[22]	validation_0-logloss:0.16120
[23]	validation_0-logloss:0.15721
[24]	validation_0-logloss:0.15229
[25]	validation_0-logloss:0.14932
[26]	validation_0-logloss:0.14525
[27]	validation_0-logloss:0.14190
[28]	validation_0-logloss:0.13881
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device='cuda', early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [34]:
y_test

tensor([0, 0, 1,  ..., 0, 0, 0], device='cuda:0')

In [35]:
# 6. Evaluate model
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test.to('cpu'), y_pred))
print("\nClassification Report:")
print(classification_report(y_test.to('cpu'), y_pred))

Accuracy: 0.97

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2020
           1       0.96      0.94      0.95       980

    accuracy                           0.97      3000
   macro avg       0.97      0.96      0.97      3000
weighted avg       0.97      0.97      0.97      3000



In [36]:
data_test_2_raw = pl.read_csv('data/merchant_locs_train.csv', separator=',', quote_char='"').sample(100000)

In [37]:
data_test_2_raw = data_test_2_raw.filter(pl.col('merchant_location').is_null() & (pl.col('online_offline_flag') == 'OFFLINE'))

In [38]:
data_test_2_raw['online_offline_flag'].value_counts()

online_offline_flag,count
str,u32
"""OFFLINE""",2001


In [39]:
data_test_2 = data_test_2_raw.with_columns(
    pl.col("merchant_name").map_elements(lambda x: count_digit(x)/len(x), return_dtype = pl.Float32).alias("digit_ratio"),
    pl.col("merchant_name").map_elements(lambda x: len(re.findall(r'\d+',x)), return_dtype = pl.Int32).alias("num_blocks"),
    #pl.when(pl.col("merchant_location").is_null()).then(pl.lit(0)).otherwise(pl.lit(1)).alias("location_is_known"),
    pl.when(pl.col("merchant_name").str.contains(r"((\bAFM\b)|(ALFAMART))|((\bIDM\b|INDOMARET))|(FAMILY\s?MART)|(HHB\s[A-Z0-9]{3})")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("known_offline_merchant")
)

In [40]:
data_test_2 = data_test_2.with_columns(
    pl.col("online_offline_flag").replace_strict(mapping).alias("label")
)

In [41]:
data_test_2

source,merchant_name,merchant_location,online_offline_flag,digit_ratio,num_blocks,known_offline_merchant,label
str,str,str,str,f32,i32,i32,i64
"""CREDIT_CARD""","""ALFAMART AN 1G9L GUBERNURKOTA …",null,"""OFFLINE""",0.05,2,1,0
"""CREDIT_CARD""","""IDM FQWB ACEH SINGKIL ID""",null,"""OFFLINE""",0.0,0,1,0
"""CREDIT_CARD""","""ALFAMART J185 MBL JAKARTA PUSA…",null,"""OFFLINE""",0.090909,1,1,0
"""CREDIT_CARD""","""ALFAMART BC76 WANAKERTA GARUT …",null,"""OFFLINE""",0.0625,1,1,0
"""CREDIT_CARD""","""IDM T1PU-KALIBARU TIMUR 2BEKAS…",null,"""OFFLINE""",0.058824,2,1,0
…,…,…,…,…,…,…,…
"""CREDIT_CARD""","""TOKO TIGA SARI LOMBOK TENGAHID""",null,"""OFFLINE""",0.0,0,0,0
"""CREDIT_CARD""","""IDM TERV-SETH ADJI - PLK PALAN…",null,"""OFFLINE""",0.0,0,1,0
"""CREDIT_CARD""","""IDM TOE3 SIMALUNGUN ID""",null,"""OFFLINE""",0.045455,1,1,0


In [42]:
def create_bert_feature(data):
    bert_input = bert_tokenizer(list(data), return_tensors="pt", padding=True, truncation=True)
    # Get embeddings
    with torch.no_grad():
        bert_output = bert_model(**bert_input)
    return bert_output.last_hidden_state[:, 0, :]

In [43]:
data_test_2_bert = create_bert_feature(data_test_2['merchant_name'])

In [44]:
#data_test_2 = vectorizer.transform(data_test_2_raw['merchant_name_clean'])

In [252]:
X_test_2 = torch.cat([torch.tensor(data_test_2[['digit_ratio','num_blocks',
                                                #'location_is_known', 
                                                'known_offline_merchant']].to_numpy()).to(device), 
                      data_test_2_bert
                     ], 
                     dim=1)
y_test_2 = torch.tensor(data_test_2['label'])

In [253]:
X_test_2.shape

torch.Size([2066, 315])

In [254]:
# 6. Evaluate model
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test_2)
print("Accuracy:", accuracy_score(y_test_2.to('cpu'), y_pred))
print("\nClassification Report:")
print(classification_report(y_test_2.to('cpu'), y_pred))


Accuracy: 0.9753146176185866

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2066
           1       0.00      0.00      0.00         0

    accuracy                           0.98      2066
   macro avg       0.50      0.49      0.49      2066
weighted avg       1.00      0.98      0.99      2066



/home/master/work/nn_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/master/work/nn_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/master/work/nn_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [220]:
data_test_2_raw = data_test_2_raw.with_columns(
    pl.lit(y_pred).alias('predicted_label')
)

In [204]:
data_test_2_raw.filter(pl.col('predicted_label') == 1).write_csv('wrong_prediction.csv')

Track Feature Experiments

In [16]:
import mlflow

In [17]:
feature_sets = {
    'basic':(['digit_ratio', 'num_blocks', 'known_offline_merchant']),
    'basic_with_bert':(['digit_ratio', 'num_blocks', 'known_offline_merchant'], ['bert'])
}

In [15]:
def evaluate_feature_set(X,y, feature_set_name, feature):
    with mlflow.start_run(run_name=f'feature_experiment_{feature_set_name}'):
        # Log feature set metadata
        mlflow.log_param("feature_set", feature_set_name)
        mlflow.log_param("num_features", len(features))
        mlflow.log_param("features", features)